# Pandas for Data Analytics

- [Pandas Documentation](http://pandas.pydata.org/pandas-docs/stable/) : latest version is 2.2.2 as of April 18, 2024
- [Python for Data Analysis](https://www.amazon.com/Python-Data-Analysis-Wrangling-Jupyter-dp-109810403X/dp/109810403X/ref=dp_ob_title_bk) - original book about pandas, now in 3rd edition (Wes McKinney - creator)
- [Data analysis in Python with pandas](https://www.youtube.com/watch?v=w26x-z-BdWQ) - YouTube training video from PyCon (Wes McKinney)
- [Data Analysis with Python and Pandas](https://www.udemy.com/course/data-analysis-with-pandas/?couponCode=LETSLEARNNOWPP) - Udemy training course

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# plot all graphs inline
%matplotlib inline    

print(f"Pandas version: {pd.__version__}")
print(f"numpy version {np.__version__}")

Pandas version: 2.2.2
numpy version 2.0.0


## Data Structures

## Series

A `Series` is a 1D data structure that can contain any data of the same type.  A minimal `Series` constructor looks like:
```
s = pd.Series(data)
```



but we are going to use the slightly more useful constructor: 
```
s = pd.Series(data, index, name)
```
here `data` can be lots of things, a list, an `np.ndarray`, a Python `dict` or a scalar value.  The `index` is not required, but if it is provided it must be the same length as the provided data.  If an `index` is not provided a zero based integer index is created based on the size of the data.  If input is a Python `dict` the key values are used as the index labels.

The `name` parameter is not needed but can be used to label the entire `Series`, if the named `Series` is then used to build a `DataFrame` (see below) the name will be used as the column label.

In [ ]:
N = 5
aser = pd.Series([1,2,3,4,5], index=list('abcdd'), name='A')
aser

In [ ]:
# Note: the index does not need to be unique
aser.index

A series acts much like a standard numpy array with a few additional features

In [ ]:
print(f'The length of the Series is: {len(aser)}')
print(f'The size attribute also gives the same result: {aser.size}')
print(f"You can total up the values in the series: {aser.sum()}")
print(f"  Or maybe you are in a factorial state of mind! {aser.product()}")

In [ ]:
pcser = aser.pct_change()
pcser

In [ ]:
# Series acts like an array or list for selection
aser[1:4]

In [ ]:
# Can also select using labels, kind of like a dict
aser['b':'d']

In [ ]:
# Can operate on a series just like a numpy array
aser*2 + 3

### References for efficiency

**Pandas doesn't implicitly copy any data behind your back**, the library really forces you to declare your intentions.  Series can be assigned to other variables but these are only references, using them touches the original series, similar to other Python references.  If you want a copy you need to use the `copy()` method, this has a cost that `pandas` doesn't want to pay without you agreeing.   It is similar for functions which could modify your data

In [ ]:
# Sort in descending order
aser.sort_values(ascending=False)

In [ ]:
# original series unaffected
aser

In [ ]:
# Reassign output to new variable or sort in place
aser.sort_values(ascending=False, inplace=True)
aser

### Data Alignment

In [ ]:
# Create another Series with a slightly different index, notice how we can set the type of the data
data = [np.random.randint(-200, 200) for i in range(5)]
bser = pd.Series(data, index=list('bcdef'), dtype=np.dtype('int16'), name='B')
bser

Now add the two series together:

In [ ]:
aser + bser

Data is **intrinsically** aligned on the index.  `aser` does not have a value at label `f` and `bser` has no value for label `a` thus in the sum these both get `NaN`

## DataFrame

A `DataFrame` is a 2D data structure contains one or more 1D data arrays or Series, each with its own data type.  A minimal `DataFrame` constructor looks like:
```
df = pd.DataFrame(data, columns, index)
```
where `data` can be

- a `dict` of 1D `np.ndarray`s, `list`, `dict` or `Series`: the `dict` keys are used as the column names
- a 2D `np.ndarray`
- a structured or record `np.ndarray`
- a `pd.Series`
- another `DataFrame`

The `index` is not required, but if provided it must be the same length as the provided data.  If an `index` is not provided a zero based integer index is created based on the size of the data.  The column names are taken from the `columns` input list (if provided) or the key values if a `dict` is used as the data.

In [ ]:
# Create from a dict of mixed data
df = pd.DataFrame({'A': aser, 'B': bser, 'C': [3]*6})
df

### Selection

In [ ]:
# Select a single column, result is a Series
df['B']

In [ ]:
# Select multiple columns, result is a DataFrame, NOTE: use list
df[['C', 'B']]

In [ ]:
# Select a row by label, result is a Series
df.loc['a']

In [ ]:
# Select a subset of rows, result is a DataFrame
df.iloc[3:6]

In [ ]:
# Select based on a boolean condition
df[df['A'] < 0.5]

### Assignment/Deletion 

In [ ]:
# Assign to an known column
df['C'] = 5
df

In [ ]:
# Assign to an unknown column creates a new column (at end by default)
df['D'] = df['A']*100
df

In [ ]:
# Delete a column
del df['D']
df

In [ ]:
# Dump all values out as a numpy array, widest type (float64) is chosen
npdata = df.values
print(f"Data type for array {npdata.dtype}, integer types cast to this type")
npdata


## Date/Time Functionality

In [ ]:
dt = pd.to_datetime('2024-06-30 15:00:00', utc=True)
dt

In [ ]:
# Datetime in nanoseconds 
dt.value

In [ ]:
pd.to_datetime(1465570900000000, unit='us', utc=True)

## DatetimeIndex

In [ ]:
dt_index = pd.date_range('2024-06-30 15:00:00', periods=100, freq='10s')
df = pd.DataFrame(np.random.randn(100, 4), columns=list('ABCD'), index=dt_index)
df.head()  # Only

In [ ]:
# Select based on timestamp, NOTE: includes the last timestamp
df['2024-06-30 15:01:00':'2024-06-30 15:02:00']

In [ ]:
df.loc['2024-06-30 15:01:00':'2024-06-30 15:02:00', ['C', 'B']]

## Data Analytics

### Reading Data

The first thing we need to do is get some data.  Pandas is VERY flexible and provides a plethora of ways to read data.   You can read from flat files (CSV, text, HTML, XML), databases, HDF5, Parquet, SPSS.  Pandas has you covered in reading data.   Since it is easiest I'm going to use CSV files

- Reference the [Pandas Input/Output documentation](https://pandas.pydata.org/pandas-docs/stable/reference/io.html) for more information

In [ ]:
nvidia_data_file = Path.cwd() / 'data' / 'nvidia.csv'
nvda_prices = pd.read_csv(nvidia_data_file)

In [ ]:
nvda_prices.head()

This is a pretty basic import, let's see if pandas can help us a little more.  The `read_csv` [method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) has about 30 kwargs to help so let's use a few.

In [ ]:
nvda_prices = pd.read_csv(nvidia_data_file,
                          parse_dates=['Date'], # parse dates in one (or more columns)
                          index_col='Date',
                         )
nvda_prices

In [ ]:
nvda_prices.index

In [ ]:
nvda_prices.describe()

In [ ]:
nvda_prices['Close'].plot(grid=True);

In [ ]:
nvda_prices.sort_index(ascending=False)

In [ ]:
nvda_prices.nlargest(columns=['Close'], n=5)

In [ ]:
nvda_prices.nsmallest(columns=['Close'], n=5)

## Data Filtering and Selection

In [ ]:
fifa_file = Path.cwd() / 'data' / 'fifa_players_22.csv'
players = pd.read_csv(fifa_file, low_memory=False)
players.info()

In [ ]:
# Note pandas does the best job it can to get the data type for the field right
print(f"Player age column type: {players['age'].dtype}")
print(f"Player name column type: {players['short_name'].dtype}")   # object is default for string
print(f"Player gk rating column type: {players['gk'].dtype}")      # this isn't exactly what we want

In [ ]:
columns_to_select= ['short_name', 'overall', 'nationality_name', 'player_positions', 'age', 'preferred_foot', 'club_name', 'gk']
players[columns_to_select].head(10)

In [ ]:
# Lets see if we can filter thisd donw a little bit, since strikers get all the glory
center_backs_only = players['player_positions'] == 'CB'
players[center_backs_only][columns_to_select].head(10)

In [ ]:
center_backs_only = players['player_positions'] == 'CB'
right_backs_only = players['player_positions'] == 'RB'
left_backs_only = players['player_positions'] == 'LB'

In [ ]:
players[center_backs_only | right_backs_only | left_backs_only ][columns_to_select].head(10)

This is shows some of the power of boolean indexing in pandas, we create a boolean index and can combine those indices using additional boolean logic to build complicated expressions.  There is a problem though with some of this position data.  Some players play multiple positions see Messi or Neymar Jr.  If we wanted to focus on all the players for a specific position we'd need to include these.  How do we create an expression for that?

In [ ]:
cdm_players_mask = players['player_positions'].str.contains('CDM')
cdm_players = players[cdm_players_mask]
cdm_players[columns_to_select].head(10)

In [ ]:
# What if we only want the younger players say <= 25
max_age = 25
young_cdm_players = players[cdm_players_mask & (players['age'] <= max_age)]
young_cdm_players[columns_to_select].head()

In [ ]:
assert(players['gk'].str.contains('\\+').all())    # have to use `\\+` for regular expression
gk_rating_data = players['gk'].str.split('+')
np_data = np.array([[int(x), int(y)] for x, y in gk_rating_data.values])
players['gk'] = np_data[:, 0] + np_data[:, 1]

In [ ]:
# How about the best outfield player to with the highest GK rating
outfield_player_mask = players['player_positions'] != 'GK'
outfield_players = players[outfield_player_mask]
outfield_players[columns_to_select].sort_values(by=['gk'], ascending=False).head()                                                 

## Grouping

In [ ]:
grp_by_nation = players.groupby(['nationality_name'])         # NOTE: the [ ] are very important here, must be a list 

In [ ]:
len(grp_by_nation) == players[['nationality_name']].nunique()

In [ ]:
#grp_by_nation.groups  (oodles and gobs of nearly unreadable data)

In [ ]:
grp_by_nation.size().sort_values(ascending=False).head(10)

In [ ]:
top_player = grp_by_nation.first()
top_player[['short_name', 'overall', 'player_positions', 'age', 'preferred_foot', 'club_name']].head(10)

In [ ]:
# What about the 5th best player for their naition
nth_player = grp_by_nation.nth(5)
nth_player[columns_to_select].head(10)

In [ ]:
nation = 'Georgia'
nat_players = grp_by_nation.get_group((nation,))
nat_players[columns_to_select].head(10)

## Multi-Indexing


In [ ]:
multi_grp = players.groupby(['player_positions', 'nationality_name'])       
multi_grp.groups

In [ ]:
multi_grp.get_group(('CDM', :)) 

## Merging, Joining and Contatenating (OH MY....)

In [ ]:
24*2*30